# Read .dat particle file for Vutata

In [1]:
# ----------------------------------------------------------------------
#
# Copyright 2009-2020 Vutara and University of Utah. All rights reserved
#
# ----------------------------------------------------------------------

# readParticleFile.py
#
# A simple python3 script to read a binary particles.dat or gzipped
# binary particle.dat.gz file.  These files contain localized particle
# information from the Vutara SRX microscope.
#
# Usage: python3 readParticleFile.py <path to .dat or .dat.gz file>
#

import os
import sys
import gzip
import struct
import numpy as np
import pandas as pd

In [2]:
def load_data(filePath):
    print("Reading ", filePath)

    filename, fileExt = os.path.splitext(filePath)

    if fileExt == ".dat":
        with open(filePath, 'rb') as f:
            file_content = f.read()
    elif fileExt == ".gz":
        with gzip.open(filePath, 'rb') as f:
            file_content = f.read()
    else:
        print("Error: Invalid file format!")
        exit(0)

    if len(file_content) == 0:
        print("Error: Empty file!")
        exit(0)

    return file_content

In [3]:
file_path = 'particles-000.dat.gz'
file_content = load_data(file_path)

Reading  particles-000.dat.gz


In [4]:
def read_header(file_content):
    # read header

    # first 4 bytes is the number of columns
    numCols = struct.unpack("<i", file_content[:4])[0]
    print ("Found ", numCols, " columns")
    byteCurr = 4

    columnNames = []
    columnBytes = []
    columnTypes = []

    # For each column we process header info
    # 4 byte int entry for the name string length
    # variable bytes for the name string
    # 4 byte int for the number of bytes in the particle data for this column
    for c in range(numCols):
        # get length of name string in bytes
        colLen = struct.unpack("<i", file_content[byteCurr:(byteCurr+4)])[0]
        byteCurr = byteCurr + 4

        # get name string
        colName = struct.unpack("<"+str(colLen)+"s", file_content[byteCurr:(byteCurr+colLen)])[0]
        columnNames.append(colName)
        byteCurr = byteCurr + colLen

        # get particle data byte count
        colBytes = struct.unpack("<i", file_content[byteCurr:(byteCurr+4)])[0]
        columnBytes.append(colBytes)
        byteCurr = byteCurr + 4

        # infer type from number of bytes and name
        if colBytes == 1: # 1 byte is always a boolean
            columnTypes.append("<?")
        elif colBytes == 4: # 4 bytes is always an int
            columnTypes.append("<i")
        elif colBytes == 8 and colName == b'frame-timestamp': # 8 bytes can be a int64 if it is a timestamp
            columnTypes.append("<q")
        elif colBytes == 8: # typically 8 bytes is a double
            columnTypes.append("<d")
        else:
            print("Error: Detected unknown byte length for column ", colName, "!")
            exit(0)

    return numCols, byteCurr, columnNames, columnBytes, columnTypes

In [5]:
numCols, byteCurr, columnNames, columnBytes, columnTypes = read_header(file_content)

Found  34  columns


In [6]:
# print header info
print("Header read")
print("column names: ", columnNames)
print("column bytes: ", columnBytes)
print("column inferred types: ", columnTypes)

Header read
column names:  [b'image-ID', b'cycle', b'z-step', b'frame', b'accum', b'probe', b'photon-count', b'photon-count11', b'photon-count12', b'photon-count21', b'photon-count22', b'psfx', b'psfy', b'psfz', b'psf-photon-count', b'x', b'y', b'z', b'amp', b'background11', b'background12', b'background21', b'background22', b'maxResidualSlope', b'chisq', b'log-likelihood', b'accuracy', b'llr', b'time-point', b'precisionx', b'precisiony', b'precisionz', b'frame-timestamp', b'vis-probe']
column bytes:  [4, 4, 4, 4, 4, 4, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 4, 8, 8, 8, 8, 4]
column inferred types:  ['<i', '<i', '<i', '<i', '<i', '<i', '<d', '<d', '<d', '<d', '<d', '<d', '<d', '<d', '<d', '<d', '<d', '<d', '<d', '<d', '<d', '<d', '<d', '<d', '<d', '<d', '<d', '<d', '<i', '<d', '<d', '<d', '<q', '<i']


In [7]:
def read_particles(numCols, byteCurr, columnNames, columnBytes, columnTypes, file_content):
    # read particles
    print("Reading particles...")

    # collect data in one large 2D array
    pointData = [[] for i in range(numCols)]

    # Go until we run out of lines
    while byteCurr < len(file_content):
        for c in range(numCols):
            val = struct.unpack(columnTypes[c], file_content[byteCurr:(byteCurr+columnBytes[c])])[0]
            pointData[c].append(val)
            byteCurr = byteCurr + columnBytes[c]

    numPoints = len(pointData[0])
    print("Read ", numPoints, " points")
    print("Done")

    # variables you should care about
    # numCols: (int) number of columns read
    # numPoints: (int) number of points read
    # columnNames: (array of strings) byte strings that describe each column
    # pointData: (2D array mixed type) the data for each point (numCols x numPoints)

    return pointData

In [8]:
# point_data = read_particles(numCols, byteCurr, columnNames, columnBytes, columnTypes, file_content)

In [9]:
def pd_read_particles(numCols, byteCurr, columnNames, columnBytes, columnTypes, file_content):
    # read particle data into pandas table
    
    # define np.dtype for data
    # convert bytes representation for column names to string
    column_names = [a.decode('utf-8)') for a in columnNames]
    
    # create type description
    d_types = [a + str(b) for a, b in zip(columnTypes, columnBytes)]

    # replace '<d8' with '<8'
    d_types = [d if d != '<d8' else '<d' for d in d_types]
    # replace '<q8' with '<i8'
    d_types = [d if d != '<q8' else '<i8' for d in d_types]
    
    # create dtype list with tuples of column name and type
    dt = list(zip(column_names, d_types))
    
    # convert byte stream into pandas table
    np_data = np.frombuffer(file_content, dt, offset = byteCurr)
    pd_point_data = pd.DataFrame(np_data)
    
    return pd_point_data

In [10]:
pd_point_data = pd_read_particles(numCols, byteCurr, columnNames, columnBytes, columnTypes, file_content)

In [11]:
pd.set_option('display.max_columns', 500)
pd_point_data.head()

,image-ID,cycle,z-step,frame,accum,probe,photon-count,photon-count11,photon-count12,photon-count21,photon-count22,psfx,psfy,psfz,psf-photon-count,x,y,z,amp,background11,background12,background21,background22,maxResidualSlope,chisq,log-likelihood,accuracy,llr,time-point,precisionx,precisiony,precisionz,frame-timestamp,vis-probe
0,0,0,0,0,1,0,4036.368530,1562.646713,2473.721817,0.0,0.0,575.766859,-368.178201,0.742345,2342.373811,7844.992907,36254.158670,-0.742345,118.844965,120.700290,155.999757,0.0,0.0,2.029944,1377.167016,-1980.177765,3.703972,1359.751409,0,14.869643,12.630648,30.079262,0,0
1,0,0,0,0,1,0,2692.671066,1349.450058,1343.221008,0.0,0.0,-464.911833,501.351017,-914.861414,1594.249227,15539.112029,19932.309140,914.861414,153.801939,173.961993,274.590968,0.0,0.0,0.854861,410.694439,-1622.535657,8.448018,426.492823,0,118.342936,128.399277,107.591452,0,0
2,0,0,0,0,1,0,2690.476089,1529.431747,1161.044342,0.0,0.0,-301.070092,235.491671,-835.818586,1592.437695,16310.909936,20299.828642,835.818586,103.611853,184.848411,283.882428,0.0,0.0,0.252503,388.603144,-1618.623726,8.699078,395.222832,0,73.548535,81.438155,119.302217,0,0
3,0,0,0,0,1,0,3375.549606,1452.265518,1923.284088,0.0,0.0,495.329558,119.810731,-989.523687,2398.201326,16450.150911,20415.509582,989.523687,180.427539,184.297772,281.167891,0.0,0.0,0.898019,403.030465,-1628.201282,6.066591,406.070622,0,92.885423,79.868608,93.022805,0,0
4,0,0,0,0,1,0,3140.120140,1213.936623,1926.183517,0.0,0.0,-3.638519,15.418841,-387.747688,1244.246346,16325.358246,4965.921491,387.747688,50.233600,80.740045,183.404086,0.0,0.0,0.251979,658.247842,-1630.077021,5.011551,620.785473,0,30.737005,31.341973,135.298587,0,0
